## Import modules

In [1]:
# Import external modules
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

pd.options.plotting.backend = "plotly"

## Import raw data

In [2]:
import gdown

url = "https://drive.google.com/drive/folders/10fxlNGVm3xIJQLB958CU56T6UTGp_md0?usp=drive_link"
gdown.download_folder(url, quiet=True, use_cookies=False)

In [2]:
# Define used paths & filenames
raw_data_path = "data/raw/"
raw_power_filename = os.path.join(
    raw_data_path, "consommation-quotidienne-brute-regionale.csv"
)
raw_weather_filename = os.path.join(raw_data_path, "donnees-synop-essentielles-omm.csv")

In [3]:
CONFIG_POWER = {
    "Date": "Date - Heure",
    "Region": "Région",
    "Power": "Consommation brute électricité (MW) - RTE",
    "Status": "Statut - RTE",
}

CONFIG_WEATHER = {
    "Date": "Date",
    "Region": "region (name)",
    "Temperature": "Température (°C)",
    "Nebulosite": "Nebulosité totale",
}

dict_diff = {
    x: CONFIG_WEATHER[x] for x in CONFIG_WEATHER.keys() if x not in CONFIG_POWER.keys()
}

DATACONFIG = {x: CONFIG_POWER[x] for x in CONFIG_POWER.keys() if x != "Status"}
DATACONFIG.update(dict_diff)

In [4]:
class Dataloader:
    def __init__(self) -> None:
        pass

    @classmethod
    def pre_treatment(
        cls, data, config, frequency: str = "h", resampling_mode: str = "sum"
    ):
        data = data[list(config.values())]
        data[config["Date"]] = pd.to_datetime(
            data[config["Date"]].str.replace("T", " ").str.replace("+", " +")
        )
        data = (
            data.groupby([config["Date"], config["Region"]])
            .first()
            .sort_index(level=config["Date"])
        )
        data = data.unstack(level=config["Region"]).asfreq("30T").interpolate()
        data = data.resample(frequency).agg(resampling_mode)
        data = data.stack(level=config["Region"])
        return data

    @classmethod
    def load_power(cls, path: str, frequency: str = "h", resampling_mode: str = "sum"):
        df = pd.read_csv(path, sep=";")
        df = df[df[CONFIG_POWER["Status"]] == "Définitif"].drop(
            CONFIG_POWER["Status"], axis=1
        )
        config = {x: CONFIG_POWER[x] for x in CONFIG_POWER.keys()}
        config.pop("Status")
        df = cls.pre_treatment(
            data=df,
            config=config,
            frequency=frequency,
            resampling_mode=resampling_mode,
        )
        return df

    @classmethod
    def load_weather(
        cls, path: str, frequency: str = "h", resampling_mode: str = "sum"
    ):
        df = pd.read_csv(path, sep=";")
        df = cls.pre_treatment(
            data=df,
            config=CONFIG_WEATHER,
            frequency=frequency,
            resampling_mode=resampling_mode,
        )
        return df

    @classmethod
    def load_data(
        cls,
        path_power: str,
        path_weather: str,
        frequency: str = "h",
        resampling_mode: str = "sum",
    ):
        df_power = cls.load_power(
            path_power, frequency=frequency, resampling_mode=resampling_mode
        )
        df_weather = cls.load_weather(
            path_weather, frequency=frequency, resampling_mode=resampling_mode
        )
        df_tot = (
            df_power.reset_index()
            .merge(
                df_weather,
                left_on=[CONFIG_POWER["Date"], CONFIG_POWER["Region"]],
                right_on=[CONFIG_WEATHER["Date"], CONFIG_WEATHER["Region"]],
            )
            .set_index([DATACONFIG["Date"], DATACONFIG["Region"]])
        )
        del df_power
        del df_weather
        return df_tot

In [5]:
df_tot = Dataloader.load_data(
    path_power=raw_power_filename, path_weather=raw_weather_filename
)

/var/folders/q9/031bbh9935n746cd0kcr5msh0000gn/T/ipykernel_9752/2547949385.py:10: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data[config["Date"]] = pd.to_datetime(
/var/folders/q9/031bbh9935n746cd0kcr5msh0000gn/T/ipykernel_9752/2547949385.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[config["Date"]] = pd.to_datetime(
/var/folders/q9/031bbh9935n746cd0kcr5msh0000gn/T/ipykernel_9752/2547949385.py:10: FutureWarning: In a future version of pandas, parsing datetimes 

## Define feature eng functions

In [6]:
# Temporal Component Extraction
# Capture temporal patterns (time of day, day of week) influencing energy consumption

# Exemple of function


def extract_time_features(df):
    # Convert 'Date - Time' index to time column
    df.reset_index(inplace=True)
    df["Date - Heure"] = pd.to_datetime(df["Date - Heure"])

    # Time component extraction
    df["heure"] = df["Date - Heure"].dt.hour
    df["jour"] = df["Date - Heure"].dt.day
    df["mois"] = df["Date - Heure"].dt.month
    df["jour_de_la_semaine"] = df["Date - Heure"].dt.dayofweek
    df["est_weekend"] = df["jour_de_la_semaine"].isin([5, 6]).astype(int)

    # Reset 'Date - Time' and 'Region' as indexes if necessary
    df.set_index(["Date - Heure", "Région"], inplace=True)
    return df

In [7]:
# Standardization
# put temperature data on a common scale for fair comparison

# Exemple of function


def normalize_temperature(df):
    scaler = MinMaxScaler()
    temp_values = df["Température (°C)"].values.reshape(-1, 1)
    df["Température_norm"] = scaler.fit_transform(temp_values)
    return df

In [8]:
# Categorization
# Transform continuous data into categories to simplify impact of cloudiness on energy consumption analysis

# Exemple of functions


def categorize_cloudiness(df):
    bins = [0, 50, 100, 150, 200, 250]
    labels = ["Très Clair", "Clair", "Partiellement Nuageux", "Nuageux", "Très Nuageux"]
    df["Nebulosité_cat"] = pd.cut(df["Nebulosité totale"], bins=bins, labels=labels)
    # One-hot encoding for categorical variables (e.g. categorized cloud cover)
    df = pd.get_dummies(df, columns=["Nebulosité_cat"])
    return df


def high_consumption_indicator(df, threshold=5000):
    df["Haute_Consommation"] = (
        df["Consommation brute électricité (MW) - RTE"] > threshold
    ).astype(int)
    return df

In [9]:
# Lag feature => How past energy consumption can influence future forecasts
# Capturing Temporal Dynamics: Lag features enable the model to understand and integrate the impact of past conditions and behaviors on current energy consumption forecasts.


def create_extended_lag_features(df):
    df.reset_index(inplace=True)
    df.sort_values(by=["Date - Heure", "Région"], inplace=True)
    # Define the different lag intervals in hours
    lag_intervals = {
        "day_1": 24,  # 1 day
        "day_7": 24 * 7,  # 7 days
        "week_1": 24 * 7,  # 1 week
        "week_4": 24 * 28,  # 4 weeks
        "year_1": 24 * 365,  # 1 year
    }
    # Create lag features for each interval
    for lag_name, lag_hours in lag_intervals.items():
        df[f"Consumption_lag_{lag_name}"] = df.groupby("Région")[
            "Consommation brute électricité (MW) - RTE"
        ].shift(periods=lag_hours)
    df.set_index(["Date - Heure", "Région"], inplace=True)
    # Drop rows with any NaN values generated due to shifting
    df.dropna(inplace=True)

    return df

## Apply the feature eng fuctions

In [10]:
df_clean = df_tot.copy()
df_clean = extract_time_features(df_clean)
df_clean = normalize_temperature(df_clean)
df_clean = categorize_cloudiness(df_clean)
df_clean = high_consumption_indicator(df_clean)
df_clean = create_extended_lag_features(df_clean)

In [11]:
df_clean

Consommation brute électricité (MW) - RTE  \
Date - Heure              Région                                                                  
2014-01-01 00:00:00+01:00 Auvergne-Rhône-Alpes                                          17172.0   
                          Bourgogne-Franche-Comté                                        5070.0   
                          Bretagne                                                       6667.0   
                          Centre-Val de Loire                                            5135.0   
                          Grand Est                                                     10604.0   
...                                                                                         ...   
2019-12-31 23:00:00+01:00 Nouvelle-Aquitaine                                            12110.0   
                          Occitanie                                                     10984.0   
                          Pays de la Loire                                               8133.0   
                          Provence-Alpes-Côte d'Azur                                    12127.0   
                          Île-de-France                                                 21295.0   

                                                      Température (°C)  \
Date - Heure              Région                                         
2014-01-01 00:00:00+01:00 Auvergne-Rhône-Alpes               16.900000   
                          Bourgogne-Franche-Comté            11.000000   
                          Bretagne                           15.700000   
                          Centre-Val de Loire                12.650000   
                          Grand Est                          12.350000   
...                                                                ...   
2019-12-31 23:00:00+01:00 Nouvelle-Aquitaine                 16.266667   
                          Occitanie                          10.000000   
                          Pays de la Loire                   15.050000   
                          Provence-Alpes-Côte d'Azur          8.900000   
                          Île-de-France                      -2.150000   

                                                      Nebulosité totale  \
Date - Heure              Région                                          
2014-01-01 00:00:00+01:00 Auvergne-Rhône-Alpes               200.000000   
                          Bourgogne-Franche-Comté            200.000000   
                          Bretagne                           150.000000   
                          Centre-Val de Loire                105.000000   
                          Grand Est                          185.000000   
...                                                                 ...   
2019-12-31 23:00:00+01:00 Nouvelle-Aquitaine                 200.000000   
                          Occitanie                          200.000000   
                          Pays de la Loire                   200.833333   
                          Provence-Alpes-Côte d'Azur          52.500000   
                          Île-de-France                       84.166667   

                                                      heure  jour  mois  \
Date - Heure              Région                                          
2014-01-01 00:00:00+01:00 Auvergne-Rhône-Alpes            0     1     1   
                          Bourgogne-Franche-Comté         0     1     1   
                          Bretagne                        0     1     1   
                          Centre-Val de Loire             0     1     1   
                          Grand Est                       0     1     1   
...                                                     ...   ...   ...   
2019-12-31 23:00:00+01:00 Nouvelle-Aquitaine             23    31    12   
                          Occitanie                      23    31    12   
                          Pays de la Loire               23    31    12   
  

In [12]:
df_clean.to_csv("clean-data-consolidated.csv")

In [13]:
df_clean

Consommation brute électricité (MW) - RTE  \
Date - Heure              Région                                                                  
2014-01-01 00:00:00+01:00 Auvergne-Rhône-Alpes                                          17172.0   
                          Bourgogne-Franche-Comté                                        5070.0   
                          Bretagne                                                       6667.0   
                          Centre-Val de Loire                                            5135.0   
                          Grand Est                                                     10604.0   
...                                                                                         ...   
2019-12-31 23:00:00+01:00 Nouvelle-Aquitaine                                            12110.0   
                          Occitanie                                                     10984.0   
                          Pays de la Loire                                               8133.0   
                          Provence-Alpes-Côte d'Azur                                    12127.0   
                          Île-de-France                                                 21295.0   

                                                      Température (°C)  \
Date - Heure              Région                                         
2014-01-01 00:00:00+01:00 Auvergne-Rhône-Alpes               16.900000   
                          Bourgogne-Franche-Comté            11.000000   
                          Bretagne                           15.700000   
                          Centre-Val de Loire                12.650000   
                          Grand Est                          12.350000   
...                                                                ...   
2019-12-31 23:00:00+01:00 Nouvelle-Aquitaine                 16.266667   
                          Occitanie                          10.000000   
                          Pays de la Loire                   15.050000   
                          Provence-Alpes-Côte d'Azur          8.900000   
                          Île-de-France                      -2.150000   

                                                      Nebulosité totale  \
Date - Heure              Région                                          
2014-01-01 00:00:00+01:00 Auvergne-Rhône-Alpes               200.000000   
                          Bourgogne-Franche-Comté            200.000000   
                          Bretagne                           150.000000   
                          Centre-Val de Loire                105.000000   
                          Grand Est                          185.000000   
...                                                                 ...   
2019-12-31 23:00:00+01:00 Nouvelle-Aquitaine                 200.000000   
                          Occitanie                          200.000000   
                          Pays de la Loire                   200.833333   
                          Provence-Alpes-Côte d'Azur          52.500000   
                          Île-de-France                       84.166667   

                                                      heure  jour  mois  \
Date - Heure              Région                                          
2014-01-01 00:00:00+01:00 Auvergne-Rhône-Alpes            0     1     1   
                          Bourgogne-Franche-Comté         0     1     1   
                          Bretagne                        0     1     1   
                          Centre-Val de Loire             0     1     1   
                          Grand Est                       0     1     1   
...                                                     ...   ...   ...   
2019-12-31 23:00:00+01:00 Nouvelle-Aquitaine             23    31    12   
                          Occitanie                      23    31    12   
                          Pays de la Loire               23    31    12   
  